In [ ]:
import os
os.environ["OPENBLAS_NUM_THREADS"] = "4"
os.environ["NUM_THREADS"] = "4"
os.environ["OMP_NUM_THREADS"] = "4"
from keras.layers import Input, Dense, Flatten, Reshape
from sklearn.feature_extraction import image as sk_image
from concurrent.futures import ProcessPoolExecutor
import cartopy.feature as cfeature
from keras.models import Model
import xarray as xr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from sklearn.cluster import AgglomerativeClustering
from scipy.signal import convolve2d 
from scipy import ndimage
from sklearn.cluster import KMeans
import numpy as np
import tensorflow as tf
from tensorflow import keras    
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
from sklearn.cluster import DBSCAN
from pyhdf.SD import SD, SDC
import matplotlib as mpl
#tf.config.threading.set_inter_op_parallelism_threads(1)
from extract_training_data import *
from sklearn.feature_extraction.image import extract_patches_2d, reconstruct_from_patches_2d
from pyhdf.error import HDF4Error
from functions import *
from tensorflow.keras.models import load_model
from sklearn.cluster import KMeans, MiniBatchKMeans
import joblib
import plot_functions
import importlib 
importlib.reload(plot_functions)
from plot_functions import *
# Visualize the result


In [ ]:
import pickle
with open('training_history_landmask_150k_1e3.pkl', 'rb') as f:
    loaded_history = pickle.load(f)

with open('training_history_landmask_1e-4_150k.pkl', 'rb') as f:
    loaded_history_2 = pickle.load(f)
with open('training_history_landmask_150k.pkl', 'rb') as f:
    loaded_history_3 = pickle.load(f)

plt.plot(loaded_history["loss"][30:150], "b", label="1e-3 loss")
plt.plot(loaded_history["val_loss"][30:150], "b--", label="1e-3 val loss")

plt.plot(loaded_history_2["loss"][30:150], "r", label="1e-4 loss")
plt.plot(loaded_history_2["val_loss"][30:150], "r--", label="1e-4 val loss")

plt.plot(loaded_history_3["loss"][30:150], "k", label="scheduler loss")
plt.plot(loaded_history_3["val_loss"][30:150], "k--", label="scheduler val loss")

plt.legend()

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,"  , len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)


#bands = [6, 7, 20, 28, 28, 31]
bands=[29]
#bands=[1]
folder = "/scratch/fslippe/modis/MOD02/daytime_1km/ /scratch/fslippe/modis/MOD02/boundary_1km/ /scratch/fslippe/modis/MOD02/night_1km/"


In [ ]:
import importlib
import extract_training_data
importlib.reload(extract_training_data)
from extract_training_data import *
#folder = "/scratch/fslippe/modis/MOD02/daytime_1km/"# /scratch/fslippe/modis/MOD02/boundary_1km/"# /scratch/fslippe/modis/MOD02/night_1km/"

start = "20201201"
end = "20210430"
# start = "20210401"
# end = "20210430"


start_converted = convert_to_day_of_year(start)
end_converted = convert_to_day_of_year(end)
print(start_converted)
print(end_converted)
x, dates, masks = extract_1km_data(folder, bands=bands, start_date=start_converted, end_date=end_converted)
x, dates, masks = zip(*[(xi, date, mask) for xi, date, mask in zip(x, dates, masks) if (xi.shape[0] > 64) and (xi.shape[1] > 64)])
x = list(x)
dates = list(dates)

#x = extract_250m_data(folder, bands=[1], start_date=start_converted, end_date=end_converted)
len(masks)




In [ ]:
import autoencoder
import importlib
importlib.reload(autoencoder)
from autoencoder import SobelFilterLayer, SimpleAutoencoder
patch_size = 64
#normalized_patches = np.concatenate([autoencoder.extract_patches(n_d) for n_d in normalized_data], axis=0)

In [ ]:
from autoencoder import SobelFilterLayer, SimpleAutoencoder
patch_size = 64
print(len(bands))
autoencoder_predict = SimpleAutoencoder(len(bands), patch_size, patch_size)

#encoder = load_model("/uio/hume/student-u37/fslippe/data/models/winter_2020_21_band(6,20,29)_encoder")
encoder = load_model("/uio/hume/student-u37/fslippe/data/models/winter_2020_21_dnb_landmask_150k_scheduler_band(29)_filter_encoder")
max_vals = np.load("/uio/hume/student-u37/fslippe/data/models/winter_2020_21_dnb_landmask_band(29)_max_vals.npy")


In [ ]:
val_data = np.load("/scratch/fslippe/modis/MOD02/test_data/normalized_testpatches_dnb_landmask_150k_band(29)_winter20_21.npy")
encoded_patches = encoder.predict(val_data)
encoded_patches_flat = encoded_patches.reshape(encoded_patches.shape[0], -1)
gc.collect()

In [ ]:
#cluster = MiniBatchKMeans(11, batch_size=32, random_state=42).fit(encoded_patches_flat)
#joblib.dump(cluster, '/uio/hume/student-u37/fslippe/data/models/winter_2020_21_dnb_landmask_150k_scheduler_band(29)_filter_cluster.pkl')

In [ ]:
### EXTRACT CAO AND NOn CAO CASES
import importlib
import extract_training_data
importlib.reload(extract_training_data)
from extract_training_data import *
# start = "20230303"
# end = "20230306"
start = "20210101"
end = "20210401"
#start = "20210701"
#end = "20210702"
start_converted = convert_to_day_of_year(start)
end_converted = convert_to_day_of_year(end)
#x_cao, dates_cao, masks_cao = extract_1km_data("/scratch/fslippe/modis/MOD02/july_2021/", bands=bands, start_date=start_converted, end_date=end_converted)

# x_cao, dates_cao, masks_cao, lon_lats = extract_1km_data("/scratch/fslippe/modis/MOD02/cao_test_data/",
#                                                          bands=bands,
#                                                          start_date=start_converted,
#                                                          end_date=end_converted,
#                                                          return_lon_lat=True)
x_cao, dates_cao, masks_cao, lon_lats = extract_1km_data(folder,
                                                         bands=bands,
                                                         start_date=start_converted,
                                                         end_date=end_converted,
                                                         return_lon_lat=True)
x_cao, dates_cao, masks_cao, lon_lats = zip(*[(xi, date, mask, lon_lat) for xi, date, mask, lon_lat in zip(x_cao, dates_cao, masks_cao, lon_lats) if (xi.shape[0] > 64) and (xi.shape[1] > 64)])

x_cao = list(x_cao)
dates_cao = list(dates_cao)


In [ ]:
#### EXTRACTING AND ENCODING PATCHES + SAVING indexes of belonging files
import functions
importlib.reload(functions)
from functions import *
cluster_map_cao = []
patches_cao, all_lon_patches_cao, all_lat_patches_cao, starts_cao, ends_cao, shapes_cao, n_patches_tot_cao, indices = generate_patches(x_cao,
                                                                                                                                       masks_cao,
                                                                                                                                       lon_lats,
                                                                                                                                       max_vals,
                                                                                                                                       autoencoder_predict)


gc.collect()

In [ ]:
index_list = [0,1,2, 3, 4, 5, 6, 7, 8, 9,10,11,12,13,14,15,16,17,18,19]
#index_list = range(len(x_cao))
#index_list = [2]

cluster_predict = cluster.predict(encoded_patches_flat_cao)
labels = cluster_predict#.labels_
cluster_map_cao = []

global_min = np.min([np.min(cm) for cm in cluster.labels_])
global_max = np.max([np.max(cm) for cm in cluster.labels_])+2
print(global_max)
plot_img_cluster_mask(x_cao,
                      labels,
                      masks_cao,
                      starts_cao,
                      ends_cao,
                      shapes_cao,
                      indices,
                      dates_cao,
                      n_patches_tot_cao,
                      patch_size,
                      global_min,
                      global_max,
                      index_list,
                      save=None)


In [ ]:
march_idx = np.where(np.array(dates_cao) == str(convert_to_day_of_year("20210301")))[0][0]
projection = ccrs.Stereographic(central_latitude=90)

x_grid, y_grid, counts = generate_hist_map(n_patches_tot_cao,
                                        indices,
                                        labels,
                                        starts_cao,
                                        ends_cao,  
                                        shapes_cao,
                                        all_lon_patches_cao,
                                        all_lat_patches_cao,  
                                        dates_cao,
                                        desired_label=2,
                                        size_threshold=15,
                                        patch_size=patch_size,
                                        global_max=global_max,
                                        projection=projection,
                                        grid_resolution = 100e3)



In [ ]:
# Visualize the result

tot_days = int(dates_cao[-1]) - int(dates_cao[0])

fig, ax = plt.subplots(subplot_kw={'projection': projection}, figsize=(15, 8), dpi=200)
plt.title("Percentage of time with predicted CAO in Jan/Feb")
ax.set_extent([-35, 35, 55, 84], ccrs.PlateCarree())  # Set extent to focus on the Arctic
ax.add_feature(cfeature.LAND, edgecolor='black')
ax.add_feature(cfeature.OCEAN)
ax.add_feature(cfeature.COASTLINE)
c = ax.contourf(x_grid, y_grid, counts/tot_days*100, transform=projection, levels=10, cmap="turbo")
plt.colorbar(c, ax=ax, orientation='vertical', label='[%]')
ax.gridlines(draw_labels=True, dms=True, x_inline=False, y_inline=False)
plt.show()
